<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/repositorios_tese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dependências

In [ ]:
!sudo apt update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [494 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [984 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,010 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [909 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages

In [ ]:
!sudo apt -y install cloc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libalgorithm-diff-perl libalgorithm-diff-xs-perl libb-hooks-endofscope-perl
  libb-hooks-op-check-perl libclass-method-modifiers-perl
  libclass-xsaccessor-perl libdevel-callchecker-perl
  libdynaloader-functions-perl libimport-into-perl
  libmodule-implementation-perl libmodule-runtime-perl libmoo-perl
  libnamespace-clean-perl libpackage-stash-perl libpackage-stash-xs-perl
  libparallel-forkmanager-perl libparams-classify-perl libregexp-common-perl
  librole-tiny-perl libsub-exporter-progressive-perl libsub-identify-perl
  libsub-name-perl libsub-quote-perl libtry-tiny-perl libvariable-magic-perl
Suggested packages:
  libscalar-number-perl
The following NEW packages will be installed:
  cloc libalgorithm-diff-perl libalgorithm-diff-xs-perl
  libb-hooks-endofscope-perl libb-hooks-op-check-perl
  libclass-method-modifiers-perl libclass-

# Clona o piloto e faz ajustes dos locais dos scripts

In [ ]:
!git clone https://github.com/armandossrecife/piloto.git

Cloning into 'piloto'...
remote: Enumerating objects: 795, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 795 (delta 99), reused 66 (delta 57), pack-reused 657
Receiving objects: 100% (795/795), 49.99 MiB | 30.34 MiB/s, done.
Resolving deltas: 100% (369/369), done.


In [ ]:
!pwd

/content


In [ ]:
!mv piloto/* /content
!ls /content

analisa_metricas.py		  extracao.py	requirements.txt
analise_dependencias_arquivos.py  gera_dsm.py	resultados
carrega_dataframes.py		  graficos	sample_data
criticos			  logs		scatter_plots.py
dao.py				  notebooks	tese
descreve_repositorio.py		  piloto	teste_dje.py
designite			  principal.py	teste_dsm.py
docs				  quartis	teste.py
dsm				  README.md	utilidades.py


In [ ]:
!rm -rf piloto

In [ ]:
!pip3 install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


In [ ]:
import os
import subprocess
import requests
import threading
import datetime
from git import Repo

def get_repository_name(url):
  repository_name = os.path.splitext(os.path.basename(url))[0]
  return repository_name

class RepoCloner:
    def __init__(self, qtd_repositorios, lista_urls):
        self.qtd_repositorios = qtd_repositorios
        self.lista_urls = lista_urls
        self.lista_threads = []
        self.lista_tempo_inicio = []
        self.lista_tempo_conclusao = []

    def my_clone(self, git_url, repo_dir):
        try:
            Repo.clone_from(git_url, repo_dir)
            tempo = datetime.datetime.now()
            print(f'Clonagem do repo {git_url} concluída com sucesso! -> instante: {tempo}')
            elemento = (repo_dir, tempo)
            self.lista_tempo_conclusao.append(elemento)
        except Exception as ex:
            print(f"Erro ao clonar o repo: {git_url} - {str(ex)}")

    def clona_repositorios(self):
        try:
          for i in range(self.qtd_repositorios):
              nome = get_repository_name(url=self.lista_urls[i])
              tdi = threading.Thread(target=self.my_clone, args=(self.lista_urls[i], nome))
              tdi.name = 'Thread-'+str(i+1)
              inicio_thread = datetime.datetime.now()
              print(f"Thread {i+1} iniciada em {inicio_thread} - clonando {nome}:{self.lista_urls[i]}...")
              self.lista_threads.append(tdi)
              elemento = (nome, inicio_thread)
              self.lista_tempo_inicio.append(elemento)
              tdi.start()

          for each in self.lista_threads:
              each.join()
        except Exception as ex:
          print(f'Erro no clona_repositorio: {str(ex)}')

    def mostra_tempo_clonagens(self):
        for t1 in self.lista_tempo_inicio:
            for t2 in self.lista_tempo_conclusao:
                if t1[0] == t2[0]:
                    print(f"Tempo de clonagem do {t1[0]} é: {t2[1]-t1[1]} s")

In [ ]:
import subprocess

def get_tag_for_commit(diretorio, commit_hash):
  command = ["git", "describe", commit_hash]
  try:
    executa = subprocess.run(command, cwd=diretorio, stdout=subprocess.PIPE, text=True)
    tag = executa.stdout.strip()
    return tag
  except subprocess.CalledProcessError as e:
    print(f'Erro: {str(e)}')
    return None

def execute_bash_command(command):
    try:
        result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            return result.stdout.strip()
        else:
            print("Error:", result.stderr.strip())
            return None
    except Exception as e:
        print("Error:", str(e))
        return None

def list_only_core(output_find):
  my_list = []
  try:
    for each in output_find.split('\n'):
      if 'examples' not in each:
        my_list.append(each)
  except Exception as ex:
    print(f'Erro list_only_core {str(ex)}')
  return my_list

# Análise dos respositórios

In [ ]:
dict_repositorios = {
	'cassandra': 'https://github.com/apache/cassandra.git',
	'kafka':'https://github.com/apache/kafka.git',
	'activemq':'https://github.com/apache/activemq.git',
	'guava':'https://github.com/google/guava.git',
	'accumulo':'https://github.com/apache/accumulo.git',
	'jackson-databind':'https://github.com/FasterXML/jackson-databind.git'
}

In [ ]:
for k, v in dict_repositorios.items():
  print(k,v)

cassandra https://github.com/apache/cassandra.git
kafka https://github.com/apache/kafka.git
activemq https://github.com/apache/activemq.git
guava https://github.com/google/guava.git
accumulo https://github.com/apache/accumulo.git
jackson-databind https://github.com/FasterXML/jackson-databind.git


In [ ]:
lista_repositorios = list(dict_repositorios.values())
print(lista_repositorios)

['https://github.com/apache/cassandra.git', 'https://github.com/apache/kafka.git', 'https://github.com/apache/activemq.git', 'https://github.com/google/guava.git', 'https://github.com/apache/accumulo.git', 'https://github.com/FasterXML/jackson-databind.git']


## Clona os repositórios localmente

In [ ]:
qtd_repositorios = len(lista_repositorios)
print(f'Clonando os {qtd_repositorios} repositórios. Aguarde...')
cloner = RepoCloner(qtd_repositorios=len(lista_repositorios), lista_urls=lista_repositorios)
cloner.clona_repositorios()
cloner.mostra_tempo_clonagens()
print(f'Clonagem de todos os {qtd_repositorios} concluída com sucesso!')

Clonando os 6 repositórios. Aguarde...
Thread 1 iniciada em 2023-08-30 19:13:22.727544 - clonando cassandra:https://github.com/apache/cassandra.git...
Thread 2 iniciada em 2023-08-30 19:13:22.733420 - clonando kafka:https://github.com/apache/kafka.git...
Thread 3 iniciada em 2023-08-30 19:13:22.734346 - clonando activemq:https://github.com/apache/activemq.git...
Thread 4 iniciada em 2023-08-30 19:13:22.751271 - clonando guava:https://github.com/google/guava.git...
Thread 5 iniciada em 2023-08-30 19:13:22.754187 - clonando accumulo:https://github.com/apache/accumulo.git...
Thread 6 iniciada em 2023-08-30 19:13:22.766215 - clonando jackson-databind:https://github.com/FasterXML/jackson-databind.git...
Clonagem do repo https://github.com/apache/activemq.git concluída com sucesso! -> instante: 2023-08-30 19:14:12.585647
Clonagem do repo https://github.com/FasterXML/jackson-databind.git concluída com sucesso! -> instante: 2023-08-30 19:14:22.652640
Clonagem do repo https://github.com/apache/

In [ ]:
!echo "Cassandra: " && ls cassandra && echo ""
!echo "Kafka:" && ls kafka && echo ""
!echo "ActiveMQ:" && ls activemq && echo ""
!echo "Guava:" && ls guava && echo ""
!echo "Accumulo:" && ls accumulo && echo ""
!echo "jackson-databind"&& ls jackson-databind && echo ""

Cassandra: 
bin			   debian	pylib
build.properties.default   doc		README.asc
build-shaded-dtest-jar.sh  examples	redhat
build.xml		   ide		relocate-dependencies.pom
CASSANDRA-14092.txt	   lib		src
CHANGES.txt		   LICENSE.txt	test
conf			   NEWS.txt	TESTING.md
CONTRIBUTING.md		   NOTICE.txt	tools

Kafka:
bin		 gradle		    log4j-appender	      settings.gradle
build.gradle	 gradle.properties  metadata		      shell
checkstyle	 gradlew	    NOTICE		      storage
clients		 gradlewAll	    NOTICE-binary	      streams
config		 group-coordinator  PULL_REQUEST_TEMPLATE.md  tests
connect		 HEADER		    raft		      tools
CONTRIBUTING.md  Jenkinsfile	    README.md		      trogdor
core		 jmh-benchmarks     release_notes.py	      vagrant
doap_Kafka.rdf	 kafka-merge-pr.py  release.py		      Vagrantfile
docs		 LICENSE	    retry_zinc		      wrapper.gradle
examples	 LICENSE-binary     reviewers.py
generator	 licenses	    server-common

ActiveMQ:
activemq-all		 activemq-log4j-appender      activemq-web
active

## Identifica os últimos 1000 commits

In [ ]:
lista_ultimos_1000_commits_c = !cd cassandra && git log -n 1000 --format="%H,%cI"
print(f'Cassandra: primeiro*: {lista_ultimos_1000_commits_c[-1]}, ultimo: {lista_ultimos_1000_commits_c[0]}')
tag_cassandra_1 = get_tag_for_commit('cassandra', lista_ultimos_1000_commits_c[-1].split(',')[0])
tag_cassandra_u = get_tag_for_commit('cassandra', lista_ultimos_1000_commits_c[0].split(',')[0])
print(f'Cassandra: tag do primeiro*: {tag_cassandra_1}, tag do ultimo: {tag_cassandra_u}')
print('')

lista_ultimos_1000_commits_k = !cd kafka && git log -n 1000 --format="%H,%cI"
print(f'Kafka: primeiro*: {lista_ultimos_1000_commits_k[-1]}, ultimo: {lista_ultimos_1000_commits_k[0]}')
tag_kafka_1 = get_tag_for_commit('kafka', lista_ultimos_1000_commits_k[-1].split(',')[0])
tag_kafka_u = get_tag_for_commit('kafka',lista_ultimos_1000_commits_k[0].split(',')[0])
print(f'Kafka: tag do primeiro*: {tag_kafka_1}, tag do ultimo: {tag_kafka_u}')
print('')

lista_ultimos_1000_commits_amq = !cd activemq && git log -n 1000 --format="%H,%cI"
print(f'ActiveMQ: primeiro*: {lista_ultimos_1000_commits_amq[-1]}, ultimo: {lista_ultimos_1000_commits_amq[0]}')
tag_amq_1 = get_tag_for_commit('activemq', lista_ultimos_1000_commits_amq[-1].split(',')[0])
tag_amq_u = get_tag_for_commit('activemq', lista_ultimos_1000_commits_amq[0].split(',')[0])
print(f'ActiveMQ: tag do primeiro*: {tag_amq_1}, tag do ultimo: {tag_amq_u}')
print('')

lista_ultimos_1000_commits_g = !cd guava && git log -n 1000 --format="%H,%cI"
print(f'Guava: primeiro*: {lista_ultimos_1000_commits_g[-1]}, ultimo: {lista_ultimos_1000_commits_g[0]}')
tag_g_1 = get_tag_for_commit('guava',lista_ultimos_1000_commits_g[-1].split(',')[0])
tag_g_u = get_tag_for_commit('guava',lista_ultimos_1000_commits_g[0].split(',')[0])
print(f'Guava: tag do primeiro*: {tag_g_1}, tag do ultimo: {tag_g_u}')

Cassandra: primeiro*: acda6fad688e95e0a9a34de28e3b61a775e59736,2022-11-07T14:59:03+02:00, ultimo: 4621ec77821c306de448e6826f46551d75e7ec18,2023-08-30T10:51:35-04:00
Cassandra: tag do primeiro*: cassandra-4.1-beta1-88-gacda6fad68, tag do ultimo: cassandra-4.0.11-1752-g4621ec7782

Kafka: primeiro*: c1a54671e8fc6c7daec5f5ec3d8c934be96b4989,2022-12-07T18:29:20-08:00, ultimo: 703e1d9faafbf07795261b3233ab985583f17fcb,2023-08-30T09:19:24-07:00
Kafka: tag do primeiro*: 0.8.0-beta1-10084-gc1a54671e8, tag do ultimo: 0.8.0-beta1-11083-g703e1d9faa

ActiveMQ: primeiro*: e3d20ed6abbb569773b67924b75b6e00b989d6b0,2020-02-27T16:58:44+01:00, ultimo: fc13d61714c1554433fd06becceed55dbe9ef8e5,2023-08-30T09:29:30-05:00
ActiveMQ: tag do primeiro*: activemq-5.15.0-641-ge3d20ed6a, tag do ultimo: activemq-5.18.0-110-gfc13d6171

Guava: primeiro*: 87a383e06a30c89e1e07689c2ea7a005a1379743,2019-12-26T15:45:53-05:00, ultimo: f491b8922f9dc8003ffdf0cbde110b76bcec4b6e,2023-08-29T09:57:17-07:00
Guava: tag do primeiro*: 

In [ ]:
lista_ultimos_1000_commits_accumulo = !cd accumulo && git log -n 1000 --format="%H,%cI"
print(f'accumulo: primeiro*: {lista_ultimos_1000_commits_accumulo[-1]}, ultimo: {lista_ultimos_1000_commits_accumulo[0]}')
tag_accumulo_1 = get_tag_for_commit('accumulo',lista_ultimos_1000_commits_accumulo[-1].split(',')[0])
tag_accumulo_u = get_tag_for_commit('accumulo',lista_ultimos_1000_commits_accumulo[0].split(',')[0])
print(f'accumulo: tag do primeiro*: {tag_accumulo_1}, tag do ultimo: {tag_accumulo_u}')

lista_ultimos_1000_commits_jd = !cd jackson-databind && git log -n 1000 --format="%H,%cI"
print(f'jackson-databind: primeiro*: {lista_ultimos_1000_commits_jd[-1]}, ultimo: {lista_ultimos_1000_commits_jd[0]}')
tag_jd_1 = get_tag_for_commit('jackson-databind',lista_ultimos_1000_commits_jd[-1].split(',')[0])
tag_jd_u = get_tag_for_commit('jackson-databind',lista_ultimos_1000_commits_jd[0].split(',')[0])
print(f'jackson-databind: tag do primeiro*: {tag_jd_1}, tag do ultimo: {tag_jd_u}')

accumulo: primeiro*: 8d714bb19bcf3fc4f29818fd16b04901fdeb3a99,2021-10-05T14:41:42-04:00, ultimo: 81c6ae16570ae1af23a91947b28fa2c673279b80,2023-08-29T15:27:35-04:00
accumulo: tag do primeiro*: rel/2.0.1-943-g8d714bb19b, tag do ultimo: rel/3.0.0-31-g81c6ae1657
jackson-databind: primeiro*: 09a67155ea8380c35137aceb9ec6b261da8910d3,2021-04-12T10:48:44-07:00, ultimo: ae952a65b6d972dd67beee6a26cbe49e0f2f5249,2023-08-28T21:41:04-07:00
jackson-databind: tag do primeiro*: jackson-databind-2.12.3, tag do ultimo: jackson-databind-2.15.2-164-gae952a65b


## Arquitetura básica do ActiveMQ

```bash
@startuml
!define ICONURLAWSPUMLL https://raw.githubusercontent.com/awslabs/aws-icons-for-plantuml/v16.0/dist/Compute/EC2Instance.puml

title ActiveMQ Software Architecture (C4 Model)

package "ActiveMQ" {
  [ActiveMQ Broker] as broker
  [Message Persistence] as persistence
  [Transport Connectors] as connectors
  [Security] as security
  [Network of Brokers] as network
  [Plugins & Extensions] as plugins
  [Management & Monitoring] as management
}

AppClient -down-> broker : Uses

broker -down-> persistence : Uses
broker -down-> connectors : Uses
broker -down-> security : Uses
broker -down-> network : Uses
broker -down-> plugins : Uses
broker -down-> management : Uses

package "Message Protocols" {
  [OpenWire] as openwire
  [AMQP] as amqp
  [MQTT] as mqtt
  [STOMP] as stomp
}

broker -down-> openwire : Supports
broker -down-> amqp : Supports
broker -down-> mqtt : Supports
broker -down-> stomp : Supports

package "Client Libraries" {
  [Java Client] as javaclient
  [Other Clients] as otherclients
}

javaclient -up-> broker : Connects to
otherclients -up-> broker : Connects to

@enduml
```

![Basic ActiveMQ Software Architecture Diagram](https://raw.githubusercontent.com/armandossrecife/teste/main/ActiveMS_Software_Architecture.png "ActiveMQ Software Architecture Diagram")

In [ ]:
command_find_src_activemq = 'cd activemq && find . -type d -wholename "*/src/main/java"'
output_find_src_activemq = execute_bash_command(command_find_src_activemq)

if output_find_src_activemq is not None:
    print("Directories with 'src/main/java':")
    print(type(output_find_src_activemq))
    print(output_find_src_activemq)

Directories with 'src/main/java':
<class 'str'>
./activemq-jms-pool/src/main/java
./activemq-stomp/src/main/java
./activemq-web-console/src/main/java
./activemq-http/src/main/java
./activemq-ra/src/main/java
./activemq-amqp/src/main/java
./activemq-cf/src/main/java
./activemq-pool/src/main/java
./activemq-karaf/src/main/java
./activemq-jaas/src/main/java
./activemq-log4j-appender/src/main/java
./activemq-kahadb-store/src/main/java
./activemq-web/src/main/java
./activemq-mqtt/src/main/java
./activemq-runtime-config/src/main/java
./activemq-openwire-legacy/src/main/java
./assembly/src/release/examples/mqtt/java/src/main/java
./assembly/src/release/examples/stomp/java/src/main/java
./assembly/src/release/examples/openwire/java/src/main/java
./assembly/src/release/examples/openwire/advanced-scenarios/jms-example-topic/src/main/java
./assembly/src/release/examples/openwire/advanced-scenarios/jms-example-message-browser/src/main/java
./assembly/src/release/examples/openwire/advanced-scenario

In [ ]:
list_only_core(output_find_src_activemq)

['./activemq-jms-pool/src/main/java',
 './activemq-stomp/src/main/java',
 './activemq-web-console/src/main/java',
 './activemq-http/src/main/java',
 './activemq-ra/src/main/java',
 './activemq-amqp/src/main/java',
 './activemq-cf/src/main/java',
 './activemq-pool/src/main/java',
 './activemq-karaf/src/main/java',
 './activemq-jaas/src/main/java',
 './activemq-log4j-appender/src/main/java',
 './activemq-kahadb-store/src/main/java',
 './activemq-web/src/main/java',
 './activemq-mqtt/src/main/java',
 './activemq-runtime-config/src/main/java',
 './activemq-openwire-legacy/src/main/java',
 './activemq-broker/src/main/java',
 './activemq-client/src/main/java',
 './activemq-openwire-generator/src/main/java',
 './activemq-osgi/src/main/java',
 './activemq-shiro/src/main/java',
 './activemq-spring/src/main/java',
 './activemq-tooling/activemq-maven-plugin/src/main/java',
 './activemq-tooling/activemq-perf-maven-plugin/src/main/java',
 './activemq-tooling/activemq-joram-jms-tests/src/main/java',

In [ ]:
!cloc activemq

    5216 text files.
    5167 unique files.                              
     298 files ignored.

github.com/AlDanial/cloc v 1.90  T=12.02 s (409.6 files/s, 61474.1 lines/s)
------------------------------------------------------------------------------------
Language                          files          blank        comment           code
------------------------------------------------------------------------------------
Java                               4355         103341         149567         415820
JavaScript                           31           2756           2049          17841
Maven                                55            662           1195          10511
XML                                 264           2696           5243           9815
HTML                                 83            635           1186           2329
JSP                                  29            333            430           1830
Bourne Shell                          7            290      

## Arqutetura básica do Guava


```bash
@startuml

!define CONTAINER frame

' Containers
[Collections] as CollectionsSystem
[Concurrency] as ConcurrencySystem
[Utilities] as UtilitiesSystem
[Caching] as CachingSystem
  
' Components within Systems
CollectionsSystem -> [ImmutableCollections]
CollectionsSystem -up-> [ThreadSafeCollections]
ConcurrencySystem -down-> [AtomicVariables]
ConcurrencySystem -down-> [ThreadingUtilities]
UtilitiesSystem -down-> [StringUtilities]  
UtilitiesSystem -down-> [IOUtilities]
CachingSystem -down-> [CachingFramework]
UtilitiesSystem -down-> [EventBus]
  
' System relationships
CollectionsSystem -down-> [ConcurrencySystem] : interacts
CollectionsSystem -down-> [UtilitiesSystem] : interacts
CollectionsSystem -down-> [CachingSystem] : interacts

@enduml
```

![Basic Guava Software Architecture Diagram](https://raw.githubusercontent.com/armandossrecife/teste/main/Guava_SA.png "Guava Software Architecture Diagram")

In [ ]:
command_find_src_guava = 'cd guava && find . -type d -wholename "*/src"'
output_find_src_guava = execute_bash_command(command_find_src_guava)

if output_find_src_guava is not None:
    print("Directories with 'src':")
    print(output_find_src_guava)

Directories with 'src':
./android/guava/src
./android/guava-testlib/src
./guava/src
./guava-gwt/src
./guava-testlib/src
./futures/listenablefuture1/src
./futures/failureaccess/src


In [ ]:
list_only_core(output_find_src_guava)

['./android/guava/src',
 './android/guava-testlib/src',
 './guava/src',
 './guava-gwt/src',
 './guava-testlib/src',
 './futures/listenablefuture1/src',
 './futures/failureaccess/src']

In [ ]:
!cloc guava

    3286 text files.
    1984 unique files.                              
    1319 files ignored.

github.com/AlDanial/cloc v 1.90  T=5.40 s (364.5 files/s, 101999.2 lines/s)
-------------------------------------------------------------------------------
Language                     files          blank        comment           code
-------------------------------------------------------------------------------
Java                          1909          59867         128589         356714
Maven                           14             51            248           2515
CSS                              1              3             39            449
JSON                             1              0              0            297
Gradle                           2             27             15            208
Markdown                         4             71              0            203
XML                             16            107            272            193
YAML                     

## Arquitetura básica do Accumulo

In [ ]:
command_find_src_accumulo = 'cd accumulo && find . -type d -wholename "*/src"'
output_find_src_accumulo = execute_bash_command(command_find_src_accumulo)

if output_find_src_accumulo is not None:
    print("Directories with 'src':")
    print(output_find_src_accumulo)

Directories with 'src':
./core/src
./src
./server/gc/src
./server/compactor/src
./server/compaction-coordinator/src
./server/base/src
./server/manager/src
./server/native/src
./server/monitor/src
./server/tserver/src
./hadoop-mapreduce/src
./iterator-test-harness/src
./minicluster/src
./start/src
./shell/src
./test/src
./assemble/src


In [ ]:
list_only_core(output_find_src_accumulo)

['./core/src',
 './src',
 './server/gc/src',
 './server/compactor/src',
 './server/compaction-coordinator/src',
 './server/base/src',
 './server/manager/src',
 './server/native/src',
 './server/monitor/src',
 './server/tserver/src',
 './hadoop-mapreduce/src',
 './iterator-test-harness/src',
 './minicluster/src',
 './start/src',
 './shell/src',
 './test/src',
 './assemble/src']

In [ ]:
!cloc accumulo

    2273 text files.
    2253 unique files.                              
      59 files ignored.

github.com/AlDanial/cloc v 1.90  T=6.62 s (334.4 files/s, 93436.8 lines/s)
---------------------------------------------------------------------------------
Language                       files          blank        comment           code
---------------------------------------------------------------------------------
Java                            2056          74614          78335         393154
JavaScript                        46           7256          11475          27325
CSS                                4           1807             94          10422
Maven                             17             68            366           4603
Thrift                            11            237            258           1496
Freemarker Template               16             80            256            950
Bourne Shell                      19            147            481            821
Bourne

## Arquitetura básica do jackson-databind

In [ ]:
command_find_src_jd = 'cd jackson-databind && find . -type d -wholename "*/src"'
output_find_src_jd = execute_bash_command(command_find_src_jd)

if output_find_src_jd is not None:
    print("Directories with 'src':")
    print(output_find_src_jd)

Directories with 'src':
./src


In [ ]:
list_only_core(output_find_src_jd)

['./src']

In [ ]:
!cloc jackson-databind

    1218 text files.
    1208 unique files.                              
      28 files ignored.

github.com/AlDanial/cloc v 1.90  T=5.28 s (225.6 files/s, 54814.9 lines/s)
-------------------------------------------------------------------------------
Language                     files          blank        comment           code
-------------------------------------------------------------------------------
Java                          1150          30199          54515         147655
JSON                             4              0              0          50492
Logos                            2            632           1064           2775
Markdown                         5            181              0            491
Maven                            1             31             92            428
YAML                             9             11              6            249
Bourne Shell                     4             39             62            228
DOS Batch                 